In [1]:
import pandas as pd 
import numpy as np 
import xgboost
import os
import time
import warnings
import sklearn
import seaborn as sns 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt 
import datetime
%matplotlib inline
warnings.filterwarnings('ignore')

c:\users\lance\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\lance\appdata\local\programs\python\python35\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
file=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/matrix_0.csv'
weatherfile=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/weather_data_2014-1-1~2016~9-30.xlsx'
numst='sum'

In [3]:
data=pd.read_csv(file)
weatherdata=pd.read_excel(weatherfile)
data=data.set_index('Unnamed: 0')
k=[str(x) for x in range(1,1455)]
k.append('sum')
data.columns=k
weatherdata=weatherdata.set_index('date')

def tz(df,str1):
    df['timestamp']=df.index
    df['time_stamp']=pd.DataFrame(pd.to_datetime(df['timestamp']))
    df['dow'] = df.time_stamp.apply(lambda x: x.dayofweek)
    df['doy'] = df.time_stamp.apply(lambda x: x.dayofyear)
    df['day'] = df.time_stamp.apply(lambda x: x.day)
    df['month'] = df.time_stamp.apply(lambda x: x.month)
    df['d-1']=df[str1].shift(1)
    df['d-2']=df[str1].shift(2)
    df['d_dif']=df['d-1']-df['d-2']
    weatherdata['date']=weatherdata.index
    combineddf=pd.merge(left=df,right=weatherdata,left_on='time_stamp',right_on='date',how='left')
    combineddf=combineddf.fillna(0)
    return combineddf

combineddf=tz(pd.DataFrame(data[numst]),numst)
combineddf=combineddf.set_index('time_stamp')

X=combineddf[['dow','doy','day','month','d-1','d-2','d_dif','max','min','max-min']]
Y=combineddf[[numst]]

Xtrain=X[X.index<'2016-07-01']
Ytrain=Y[Y.index<'2016-07-01']

Xtest=X[(X.index>='2016-07-01')&(X.index<'2016-08-01')]
Ytest=Y[(Y.index>='2016-07-01')&(Y.index<'2016-08-01')]

#rfr=xgboost.sklearn.XGBRegressor()
rfr=GradientBoostingRegressor(max_depth=7,n_estimators=1000,verbose=1,learning_rate=0.09,random_state=1)

In [4]:
rfr.fit(Xtrain,Ytrain)
print(rfr.score(Xtrain,Ytrain))
print(rfr.score(Xtest,Ytest))

result=pd.read_excel(r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/result.xlsx')

result['dow'] = result.date.apply(lambda x: x.dayofweek)
result['doy'] = result.date.apply(lambda x: x.dayofyear)
result['day'] = result.date.apply(lambda x: x.day)
result['month'] = result.date.apply(lambda x: x.month)

combinedresultdf=pd.merge(left=result,right=weatherdata,left_on='date',right_on='date',how='left')

combinedresultdf=combinedresultdf[['date','dow','doy','day','month','max','min','max-min']]

combinedresultdf['d-1']=0
combinedresultdf['d-2']=0
combinedresultdf['d_dif']=0

combinedresultdf=combinedresultdf.set_index('date')
combinedresultdf.head()
c2=combinedresultdf

c2=c2.reset_index()


c2.loc[1,'d_dif']=c2.loc[1,'d-1']-c2.loc[1,'d-2']


namelist=['dow','doy','day','month','d-1','d-2','d_dif','max','min','max-min']
c2.head()

      Iter       Train Loss   Remaining Time 
         1 184294470789.4765            4.00s
         2 156096807748.1225            3.50s
         3 132754068852.7268            3.33s
         4 113010033644.5057            3.24s
         5 96845604734.1499            3.09s
         6 82878129435.1915            2.98s
         7 71133238476.4604            2.98s
         8 61664895566.4083            3.04s
         9 53494893635.3270            2.98s
        10 46577049473.4033            3.02s
        20 14237215672.9956            3.14s
        30  6352469172.9053            3.04s
        40  3524576517.2601            2.64s
        50  2170923775.9591            2.31s
        60  1423050234.7955            2.05s
        70  1007993543.5417            1.91s
        80   566554704.8245            1.82s
        90   364407887.2940            1.72s
       100   246488097.0152            1.63s
       200    10971076.2809            1.23s
       300      629991.2389            0.95s
     

,date,dow,doy,day,month,max,min,max-min,d-1,d-2,d_dif
0,2016-09-01,3,245,1,9,33,23,10,0,0,0
1,2016-09-02,4,246,2,9,34,22,12,0,0,0
2,2016-09-03,5,247,3,9,34,23,11,0,0,0
3,2016-09-04,6,248,4,9,31,22,9,0,0,0
4,2016-09-05,0,249,5,9,32,23,9,0,0,0


In [5]:
c2.loc[0,'d-1']=Y.loc['2016-08-31'][0]
c2.loc[0,'d-2']=Y.loc['2016-08-30'][0]
c2.loc[0,'d_dif']=c2.loc[0,'d-1']-c2.loc[0,'d-2']
c2.loc[1,'d-2']=Y.loc['2016-08-31'][0]
c2.head()

,date,dow,doy,day,month,max,min,max-min,d-1,d-2,d_dif
0,2016-09-01,3,245,1,9,33,23,10,3664003,3607667,56336
1,2016-09-02,4,246,2,9,34,22,12,0,3664003,0
2,2016-09-03,5,247,3,9,34,23,11,0,0,0
3,2016-09-04,6,248,4,9,31,22,9,0,0,0
4,2016-09-05,0,249,5,9,32,23,9,0,0,0


In [6]:
for x in range(30):
    if x!=29:
        a=rfr.predict(pd.DataFrame(c2.loc[x,namelist]).T)[0]
        c2.loc[x+1,'d-1']=a
        c2.loc[x+2,'d-2']=a
        c2.loc[x,'d_dif']=c2.loc[x,'d-1']-c2.loc[x,'d-2']
        c2.loc[x,'pre']=a
    elif x==29:
        a=rfr.predict(pd.DataFrame(c2.loc[x,namelist]).T)[0]
        c2.loc[x,'pre']=a
        c2.loc[x,'d_dif']=c2.loc[x,'d-1']-c2.loc[x,'d-2']

c2=c2[c2.index<30]
c2

,date,dow,doy,day,month,max,min,max-min,d-1,d-2,d_dif,pre
0,2016-09-01,3.0,245.0,1.0,9.0,33.0,23.0,10.0,3.664003e+06,3.607667e+06,56336.000000,3.544920e+06
1,2016-09-02,4.0,246.0,2.0,9.0,34.0,22.0,12.0,3.544920e+06,3.664003e+06,-119082.685919,3.777701e+06
2,2016-09-03,5.0,247.0,3.0,9.0,34.0,23.0,11.0,3.777701e+06,3.544920e+06,232780.977735,3.770472e+06
3,2016-09-04,6.0,248.0,4.0,9.0,31.0,22.0,9.0,3.770472e+06,3.777701e+06,-7229.183870,3.645848e+06
4,2016-09-05,0.0,249.0,5.0,9.0,32.0,23.0,9.0,3.645848e+06,3.770472e+06,-124624.206202,4.066916e+06
5,2016-09-06,1.0,250.0,6.0,9.0,27.0,21.0,6.0,4.066916e+06,3.645848e+06,421068.245885,3.928304e+06
6,2016-09-07,2.0,251.0,7.0,9.0,27.0,20.0,7.0,3.928304e+06,4.066916e+06,-138611.663792,3.852963e+06
7,2016-09-08,3.0,252.0,8.0,9.0,30.0,21.0,9.0,3.852963e+06,3.928304e+06,-75341.364062,3.752753e+06
8,2016-09-09,4.0,253.0,9.0,9.0,33.0,23.0,10.0,3.752753e+06,3.852963e+06,-100209.783116,3.896242e+06
9,2016-09-10,5.0,254.0,10.0,9.0,30.0,21.0,9.0,3.896242e+06,3.752753e+06,143488.964612,3.431571e+06


In [161]:
filename=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/'+numst+r'.xlsx'
c2[['date','pre']].to_excel(filename)